In [1]:
import io
from IPython.nbformat import current
def execute_notebook(nbfile):
    with io.open(nbfile) as f:
        nb = current.read(f, 'json')
    ip = get_ipython()
    for cell in nb.worksheets[0].cells:
        if cell.cell_type != 'code':
            continue
        ip.run_cell(cell.input)
execute_notebook("/cellar/users/ramarty/Projects/hla_sex/bin/imports.ipynb")

/cellar/users/ramarty/anaconda/lib/python2.7/site-packages/IPython/nbformat.py:13: ShimWarning: The `IPython.nbformat` package has been deprecated since IPython 4.0. You should import from nbformat instead.
  "You should import from nbformat instead.", ShimWarning)
/cellar/users/ramarty/anaconda/lib/python2.7/site-packages/nbformat/current.py:19: UserWarning: nbformat.current is deprecated.

- use nbformat for read/write/validate public API
- use nbformat.vX directly to composing notebooks of a particular version

  """)


Populating the interactive namespace from numpy and matplotlib


In [2]:
import warnings
warnings.filterwarnings('ignore')

### Gathering all relevant clinical information <br>
Using Brian's firehose download <br>
Needed categories: Sex, Age at diagnosis and tumor type

In [77]:
categories = ['patient.samples.sample.bcr_sample_barcode', 'patient.age_at_initial_pathologic_diagnosis', 
              'patient.gender', 'admin.disease_code']
alternative_categories = ['patient.samples.sample.bcr_sample_barcode', 
                          'patient.primary_pathology.age_at_initial_pathologic_diagnosis',
                          'patient.gender', 'admin.disease_code']

In [3]:
tissues = [x for x in os.listdir('/cellar/users/btsui/Data/firehose/untar/') if x not in ['UCS','PANGI', 'FPPP']]

In [93]:
all_dfs = []
for tissue in tissues:
    try:
        tmp_df = pd.read_csv('/cellar/users/btsui/Data/firehose/untar/{0}/gdac.broadinstitute.org_{0}.Merge_Clinical.Level_1.2016071500.0.0/{0}.clin.merged.txt'.format(tissue),
               sep='\t', index_col=0).transpose()
        tmp_df = tmp_df[categories]
        tmp_df.columns = ['Barcode', 'Age', 'Sex', 'Tissue']
        all_dfs.append(tmp_df)
    except:
        tmp_df = pd.read_csv('/cellar/users/btsui/Data/firehose/untar/{0}/gdac.broadinstitute.org_{0}.Merge_Clinical.Level_1.2016071500.0.0/{0}.clin.merged.txt'.format(tissue),
               sep='\t', index_col=0).transpose()
        tmp_df = tmp_df[alternative_categories]
        tmp_df.columns = ['Barcode', 'Age', 'Sex', 'Tissue']
        all_dfs.append(tmp_df)

In [145]:
df = pd.concat(all_dfs)

In [146]:
def get_title(x):
    return x.title()
def get_upper(x):
    return x.upper()
def get_upper_shortenend(x):
    return x.upper()[:12]

In [147]:
df.Barcode = df.Barcode.apply(get_upper_shortenend)
df.Tissue = df.Tissue.apply(get_upper)
df.Sex = df.Sex.astype(str)
df.Sex = df.Sex.apply(get_title)

In [154]:
df = df.dropna().drop_duplicates()

In [157]:
df.index = list(df.Barcode)
df = df[['Age', 'Sex', 'Tissue']]

In [158]:
df.to_csv('/cellar/users/ramarty/Data/hla_sex/clean_data/clinical.csv')